<a href="https://colab.research.google.com/github/dnstjr4567/mo2023-1/blob/main/%EC%A0%84%EB%A0%A5%EC%98%88%EC%B8%A1prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import time
import pandas as pd
from keras.datasets import mnist
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_iris, make_circles, make_moons
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve, auc, f1_score, mean_squared_error, accuracy_score
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from google.colab import files
import io
from prophet import Prophet
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
building = pd.read_csv('/content/drive/MyDrive/building_info.csv', encoding = 'cp949')
test = pd.read_csv('/content/drive/MyDrive/test.csv', encoding = 'cp949')
train = pd.read_csv('/content/drive/MyDrive/train.csv', encoding = 'cp949')
sub = pd.read_csv('/content/drive/MyDrive/sample_submission.csv',encoding = 'cp949')

In [ ]:
df = pd.DataFrame()
df['ds'] = train['일시']
df['y'] = train['전력소비량(kWh)']
df['ds'] = pd.to_datetime(df['ds'], format='%Y%m%d %H')


# 데이터프레임을 2040 단위로 나누어 리스트로 저장
dfs_list = [df[i:i+2040] for i in range(0, len(df), 2040)]

# dfs_list에 저장된 데이터프레임을 df100까지만 남기기
#df100 = pd.concat(dfs_list[:100], ignore_index=True)

In [ ]:
dfs_list[1][:2040]

,ds,y
2040,2022-06-01 00:00:00,1170.36
2041,2022-06-01 01:00:00,1146.96
2042,2022-06-01 02:00:00,1115.28
2043,2022-06-01 03:00:00,1107.00
2044,2022-06-01 04:00:00,1069.92
...,...,...
4075,2022-08-24 19:00:00,1866.96
4076,2022-08-24 20:00:00,1693.80
4077,2022-08-24 21:00:00,1564.92
4078,2022-08-24 22:00:00,948.96


In [ ]:
"""m = Prophet()
m.fit(df[:])
future = m.make_future_dataframe(freq='H',periods=24*7)
future.tail()"""

"m = Prophet()\nm.fit(df[:])\nfuture = m.make_future_dataframe(freq='H',periods=24*7)\nfuture.tail()"

In [ ]:
"""forecast = m.predict(future)
forecast[['ds', 'yhat']].tail()"""

"forecast = m.predict(future)\nforecast[['ds', 'yhat']].tail()"

In [ ]:

# List to store forecasts from each model
forecasts = []

# Iterate through each data frame in dfs_list and create a Prophet model
for df_chunk in dfs_list:
    m = Prophet()
    # Fit the model to the current data frame chunk
    m.fit(df_chunk[:])

    # Create a future dataframe for the next 7 days with hourly frequency
    future = m.make_future_dataframe(freq='H', periods=24*7)

    # Make predictions for the future dataframe
    forecast = m.predict(future)

    # Append the forecast to the list of forecasts
    forecasts.append(forecast)

# Now, 'forecasts' list will contain 100 forecasts, each corresponding to one chunk of data


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsodkwok3/8rjd5aj4.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsodkwok3/ain975hy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=59567', 'data', 'file=/tmp/tmpsodkwok3/8rjd5aj4.json', 'init=/tmp/tmpsodkwok3/ain975hy.json', 'output', 'file=/tmp/tmpsodkwok3/prophet_models0kf7nx4/prophet_model-20230726014926.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:49:26 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:49:27 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/t

In [ ]:
forecasts[0][2040:]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
2040,2022-08-25 00:00:00,2823.542566,1474.013497,2269.887492,2823.542566,2823.542566,-968.561954,-968.561954,-968.561954,-1017.367350,-1017.367350,-1017.367350,48.805396,48.805396,48.805396,0.0,0.0,0.0,1854.980612
2041,2022-08-25 01:00:00,2823.182836,1452.078661,2214.563087,2823.182836,2823.182836,-1000.153072,-1000.153072,-1000.153072,-1047.779404,-1047.779404,-1047.779404,47.626332,47.626332,47.626332,0.0,0.0,0.0,1823.029764
2042,2022-08-25 02:00:00,2822.823105,1364.913457,2196.086240,2822.823105,2822.823105,-1059.264445,-1059.264445,-1059.264445,-1105.803868,-1105.803868,-1105.803868,46.539423,46.539423,46.539423,0.0,0.0,0.0,1763.558661
2043,2022-08-25 03:00:00,2822.463375,1305.568469,2103.665098,2822.463375,2822.463375,-1126.263309,-1126.263309,-1126.263309,-1171.845790,-1171.845790,-1171.845790,45.582481,45.582481,45.582481,0.0,0.0,0.0,1696.200066
2044,2022-08-25 04:00:00,2822.103645,1306.781324,2107.375863,2822.103645,2822.103645,-1145.627818,-1145.627818,-1145.627818,-1190.422670,-1190.422670,-1190.422670,44.794852,44.794852,44.794852,0.0,0.0,0.0,1676.475827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,2022-08-31 19:00:00,2764.906541,2419.822719,3280.917207,2729.411724,2795.831204,78.679703,78.679703,78.679703,23.737578,23.737578,23.737578,54.942125,54.942125,54.942125,0.0,0.0,0.0,2843.586244
2204,2022-08-31 20:00:00,2764.546811,2048.404323,2829.260680,2728.407154,2795.802532,-333.225109,-333.225109,-333.225109,-386.999715,-386.999715,-386.999715,53.774606,53.774606,53.774606,0.0,0.0,0.0,2431.321701
2205,2022-08-31 21:00:00,2764.187080,1674.954499,2530.820615,2727.690916,2795.761552,-655.584770,-655.584770,-655.584770,-708.138112,-708.138112,-708.138112,52.553342,52.553342,52.553342,0.0,0.0,0.0,2108.602310
2206,2022-08-31 22:00:00,2763.827350,1481.410060,2313.933273,2726.998018,2795.720572,-851.960809,-851.960809,-851.960809,-903.260969,-903.260969,-903.260969,51.300160,51.300160,51.300160,0.0,0.0,0.0,1911.866542


In [ ]:
# Initialize an empty list to store the results
answer = []

# Loop through the forecasts list to extract 'yhat' values from each model
for forecast in forecasts:
    yhat_values = forecast[2040:]['yhat'].tolist()
    answer.extend(yhat_values)

# 'answer' now contains all the 'yhat' values from all the models
answer[48:]

[1921.9170454840394,
 1878.6442003891661,
 1806.4868470731067,
 1725.091837013316,
 1690.0058056088812,
 1765.4299864642933,
 1993.2726740400174,
 2371.166537007318,
 2847.781073867117,
 3337.9682264223766,
 3752.8590300511473,
 4032.136299716555,
 4162.08707077457,
 4168.864036349247,
 4090.983386430451,
 3949.5117522490577,
 3737.124585716423,
 3434.158528144465,
 3039.420190048129,
 2591.384202149186,
 2161.59598932902,
 1822.437106488451,
 1610.4503299721828,
 1509.8255594161014,
 1466.5470838953804,
 1423.4078722465433,
 1354.2224653710737,
 1278.6597156348885,
 1252.26309441159,
 1339.2053234420507,
 1581.3409200697038,
 1976.223858937983,
 2472.4206388797447,
 2984.6567648346877,
 3423.9145148345374,
 3729.706979408164,
 3888.1320779878743,
 3925.135777224651,
 3879.0118968112492,
 3770.591185135837,
 3592.301970968449,
 3324.224920472078,
 2964.9042949593527,
 2552.5485424188287,
 2158.435646097169,
 1854.6810410514158,
 1677.5652640046942,
 1611.0223629168975,
 1600.7895929319

In [ ]:
sub['answer'] = answer
sub

,num_date_time,answer
0,1_20220825 00,1854.980612
1,1_20220825 01,1823.029764
2,1_20220825 02,1763.558661
3,1_20220825 03,1696.200066
4,1_20220825 04,1676.475827
...,...,...
16795,100_20220831 19,989.487761
16796,100_20220831 20,908.439229
16797,100_20220831 21,804.232654
16798,100_20220831 22,680.273344


In [ ]:
#sub.to_csv('./prophet_submission.csv', index=False)